In [1]:
import csv
import pandas as pd
import warnings

# ugly but it works

In [2]:
def parse_dates(dataframe, date_column, print_format=False):
    if pd.api.types.is_datetime64_any_dtype(dataframe[date_column]):
        return dataframe
    
    warnings.filterwarnings("error")
    date_formats = ["%m/%d/%Y", "%m/%d/%y", "%Y/%m/%d", "%Y-%m-%d"]
    correct_format = False
    for date_format in date_formats:
        try:
            dataframe[date_column] = pd.to_datetime(dataframe[date_column], format=date_format)
            correct_format = True
            if print_format:
                print(date_format)
        except (UserWarning, ValueError):
            continue

    warnings.resetwarnings()
    if not correct_format:
        raise NotImplementedError('Correct date format not found.')

    return dataframe

### Required functions to crop csv files

**METADATA at the bottom of the file, won't parse**:
[data\\2017\\S010102_VOC_2017_EN.csv](data\\2017\\S010102_VOC_2017_EN.csv)

In [3]:
def how_many_csv_rows_to_skip(filepath, date_column_list):
    num_rows_before_header = 0

    with open(filepath, 'r', encoding='ISO-8859-1') as file:
        for row in csv.reader(file):
            for date_column in date_column_list:
                if date_column in row:
                    return num_rows_before_header, date_column
            num_rows_before_header += 1

    return None, None


def csv_to_pandas(filepath, date_column_list=['Compounds', 'Sampling Date']):
    num_rows_to_skip, date_column = how_many_csv_rows_to_skip(filepath, date_column_list)
    parsed_data = []

    with open(filepath, 'r', encoding='ISO-8859-1') as file:
        reader = csv.reader(file)

        for _ in range(num_rows_to_skip):
            next(reader)
        
        headers = next(reader)
        
        for row in reader:
            if all(item == '' for item in row):
                break
            parsed_data.append(row)

    df = pd.DataFrame(parsed_data, columns=headers)

    return df

### testing
(plus some helper functions that may be useful)

In [4]:
################## CHECK IMPORT SCRIPTS ARE WORKING ##################

import openpyxl
import os
import xlrd
import io


ignore_list = [
    'data\\ddmmyyyy.xlsx', 'data\\mmddyyyy.xlsx', 'data\\yyyyddmm.xlsx', 'data\\yyyymmdd.xlsx',
    'data\\ddmmyyyy.csv', 'data\\mmddyyyy.csv', 'data\\yyyyddmm.csv', 'data\\yyyymmdd.csv',
    'data\\2006\\S62601_VOCS.csv', 'data\\2007\\S62601_VOCS.csv',    # sideways csv for some reason
    'data\\2008\\S90227_VOC.csv', 'data\\2009\\S90227_VOC.csv', 'data\\2010\\S90227_VOC.csv',  # sampling data relocated
    'data\\2015\\ChangeLog_Jan2017.xls'
]


def how_many_csv_rows_to_skip(filepath, date_column_list):
    num_rows_before_header = 0

    with open(filepath, 'r', encoding='ISO-8859-1') as file:
        for row in csv.reader(file):
            for date_column in date_column_list:
                if date_column in row:
                    return num_rows_before_header, date_column
            num_rows_before_header += 1

    return None, None


def how_many_xlsx_rows_to_skip(filepath, date_column_list):
    num_rows_before_header = 0

    with open(filepath, "rb") as f:  # fucking malloc
        in_mem_file = io.BytesIO(f.read())
    wb = openpyxl.load_workbook(in_mem_file, read_only=True)

    sheet = None
    for name in wb.sheetnames:
        if 'voc' in name.lower():
            sheet = wb[name]
            break
        if 'data' in name.lower() and 'metadata' not in name.lower():
            sheet = wb[name]
            break

    for row in sheet.iter_rows(values_only=True):
        for date_column in date_column_list:
            if date_column in row:
                return num_rows_before_header, date_column, name
        num_rows_before_header += 1
       
    return None, None, None


def get_xls_sheet(workbook):
    for name in workbook.sheet_names():
        if 'voc' in name.lower():
            return workbook[name], name
        elif 'data' in name.lower():
            return workbook[name], name
    return workbook.sheet_by_index(0), 0


def how_many_xls_rows_to_skip(filepath, date_column_list):
    num_rows_before_header = 0
    wb = xlrd.open_workbook(filepath, encoding_override='ISO-8859-1')

    sheet, name = get_xls_sheet(wb)

    for row_idx in range(sheet.nrows):
        row = sheet.row_values(row_idx)
        for date_column in date_column_list:
            if date_column in row:
                return num_rows_before_header, date_column, name
        num_rows_before_header += 1

    wb.close()

    return None, None, None


def try_all_imports(skip_xls=False, skip_xlsx=False, skip_csv=False, ignore_list=[None], filter_warnings=False):
    if filter_warnings:
        warnings.filterwarnings("error")
    n_errors, n_files, n_xls, n_xlsx, n_csv, n_xls_complete, n_xlsx_complete, n_csv_complete  = 0, 0, 0, 0, 0, 0, 0, 0
    error_list = list()

    for root, dirs, files in os.walk('data'):
        for filename in files:
            filepath = os.path.join(root, filename)
            if filepath in ignore_list:
                continue

            extension = filepath.split('.')[-1].lower()
            if skip_xls and extension == 'xls':
                continue
            elif skip_xlsx and extension == 'xlsx':
                continue
            elif skip_csv and extension == 'csv':
                continue
            elif extension not in ['xls', 'xlsx', 'csv']:
                continue

            name = filepath.split('.')[0].lower()
            if name[-2:].lower() == 'fr':
                continue
            else:
                n_files += 1
                try:
                    if extension == 'xls':
                        n_xls += 1
                        header, date_column, sheet_name = how_many_xls_rows_to_skip(filepath, ['Compounds', 'Sampling Date'])
                        workbook = xlrd.open_workbook(filepath, encoding_override='ISO-8859-1')
                        dataframe = pd.read_excel(workbook, header=header, sheet_name=sheet_name, engine='xlrd')
                        parse_dates(dataframe, date_column=date_column)
                        n_xls_complete += 1
                    elif extension == 'xlsx':
                        n_xlsx += 1
                        header, date_column, sheet_name = how_many_xlsx_rows_to_skip(filepath, ['Compounds', 'Sampling Date'])
                        parse_dates(pd.read_excel(filepath, header=header, sheet_name=sheet_name), date_column=date_column)
                        n_xlsx_complete += 1
                    elif extension == 'csv':
                        n_csv += 1
                        header, date_column = how_many_csv_rows_to_skip(filepath, ['Compounds', 'Sampling Date'])
                        parse_dates(csv_to_pandas(filepath), date_column=date_column)
                        n_csv_complete += 1
                except:
                    n_errors += 1
                    error_list.append(filepath)
                    
    if filter_warnings:
        warnings.resetwarnings()

    print(f'{n_errors} errors / {n_files} total files')
    print(f'xls: {n_xls_complete} out of {n_xls}')
    print(f'csv: {n_csv_complete} out of {n_csv}')
    print(f'xlsx: {n_xlsx_complete} out of {n_xlsx}')

    return error_list

##### XLSX
* seems to work

In [5]:
xlsx_errors = try_all_imports(skip_xls=True, skip_xlsx=False, skip_csv=True, ignore_list=ignore_list)

0 errors / 118 total files
xls: 0 out of 0
csv: 0 out of 0
xlsx: 118 out of 118


##### XLS
* seems to work

In [6]:
xls_errors = try_all_imports(skip_xls=False, skip_xlsx=True, skip_csv=True, ignore_list=ignore_list)

0 errors / 220 total files
xls: 220 out of 220
csv: 0 out of 0
xlsx: 0 out of 0


##### CSV

In [7]:
csv_errors = try_all_imports(skip_xls=True, skip_xlsx=True, skip_csv=False, ignore_list=ignore_list)

0 errors / 628 total files
xls: 0 out of 0
csv: 628 out of 628
xlsx: 0 out of 0


### Check Random Files

In [8]:
import random

def pick_random_file(directory, ignore_list):
    all_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            filepath = os.path.join(root, file)
            if (file.split('.')[0].lower()[-2:] != 'fr') and (filepath not in ignore_list):
                all_files.append(os.path.join(root, file))

    return random.choice(all_files)


In [9]:
filepath = pick_random_file('data', ignore_list)
extension = filepath.split('.')[-1]

if extension == 'xls':
    header, date_column, sheet_name = how_many_xls_rows_to_skip(filepath, ['Compounds', 'Sampling Date'])
    workbook = xlrd.open_workbook(filepath, encoding_override='ISO-8859-1')
    dataframe = pd.read_excel(workbook, header=header, sheet_name=sheet_name, engine='xlrd')
    df = parse_dates(dataframe, date_column=date_column)
elif extension == 'xlsx':
    header, date_column, sheet_name = how_many_xlsx_rows_to_skip(filepath, ['Compounds', 'Sampling Date'])
    df = parse_dates(pd.read_excel(filepath, header=header, sheet_name=sheet_name), date_column=date_column)
elif extension == 'csv':
    header, date_column = how_many_csv_rows_to_skip(filepath, ['Compounds', 'Sampling Date'])
    df = parse_dates(csv_to_pandas(filepath), date_column=date_column)

display(df.dtypes)
display(df.head(5))

NAPS ID                               int64
Sampling Date                datetime64[ns]
Sample Type                          object
Ethylene                            float64
Ethylene-MDL                        float64
                                  ...      
Dodecane-MDL                        float64
Dodecane-VFlag                       object
Hexachlorobutadiene                 float64
Hexachlorobutadiene-MDL             float64
Hexachlorobutadiene-VFlag            object
Length: 330, dtype: object

,NAPS ID,Sampling Date,Sample Type,Ethylene,Ethylene-MDL,Ethylene-VFlag,Acetylene,Acetylene-MDL,Acetylene-VFlag,Ethane,...,"1,2,4-Trichlorobenzene-VFlag",Naphthalene,Naphthalene-MDL,Naphthalene-VFlag,Dodecane,Dodecane-MDL,Dodecane-VFlag,Hexachlorobutadiene,Hexachlorobutadiene-MDL,Hexachlorobutadiene-VFlag
0,40901,2020-01-04,R,-999.000000,-999.00,M1,-999.000000,-999.00,M1,-999.000000,...,M1,-999.000000,-999.00,M1,-999.000000,-999.0,M1,-999.000000,-999.00,M1
1,40901,2020-01-10,R,0.269087,0.03,NaN,0.290962,0.01,NaN,2.764861,...,NaN,0.021826,0.07,NaN,0.017633,0.1,NaN,0.002848,0.06,NaN
2,40901,2020-01-16,R,0.573501,0.03,NaN,0.432543,0.01,NaN,4.541279,...,NaN,0.027864,0.07,NaN,0.015945,0.1,NaN,0.002526,0.06,NaN
3,40901,2020-01-22,R,0.504552,0.03,NaN,0.392287,0.01,NaN,2.563230,...,NaN,0.023531,0.07,NaN,0.009625,0.1,NaN,0.002116,0.06,NaN
4,40901,2020-01-28,R,0.200371,0.03,NaN,0.297843,0.01,NaN,2.475414,...,NaN,0.022652,0.07,NaN,0.010279,0.1,NaN,0.005572,0.06,NaN
